In [5]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader("faq.csv")
docs = loader.load()

In [6]:
docs[0]

Document(metadata={'source': 'faq.csv', 'row': 0}, page_content="Question: How can I reset my password?\nAnswer: Go to the login page and click on 'Forgot Password'. Enter your registered email address and follow the instructions sent to your inbox.")

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os 
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(api_key=api_key,model="gemini-1.5-flash-latest",temperature=0.3)

response = llm.invoke("write a two line of 8 word combined on cricket")

print(response.content)

Cricket batsmen fiercely strike, soaring sixes fly.
Bowlers spin, swing, wickets tumble, crowds wildly cry.


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [9]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(docs,embeddings)

vectordb.save_local('faiss_index')

In [10]:
loaded_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

query = "actually i forget my password, how i reset it"

results = loaded_db.similarity_search(query,k=2)

for r in results:
    print(r.page_content)

Question: How can I reset my password?
Answer: Go to the login page and click on 'Forgot Password'. Enter your registered email address and follow the instructions sent to your inbox.
Question: How do I create an account?
Answer: Click on 'Sign Up' at the top-right corner of the website and fill in your details to create an account.


In [13]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

retriever = loaded_db.as_retriever()

memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True,k=3)
prompt_template = '''You are a helpful assistant for answering FAQs.

Use ONLY the following context from the FAQ database to answer the question.
If the answer is not in the context, reply strictly with: "I don't know."

Context:
{context}

Question:
{question}

Answer:'''

QA_PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context","question"]
)
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory =memory,
    combine_docs_chain_kwargs = {"prompt":QA_PROMPT}
)

result = chain.invoke({"question":"what is samosa"})
print(result["answer"])

I don't know.
